In [1]:
import tensorflow._api.v2.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import random

tf.reset_default_graph()

NUM_CLASSES = 20


class MLP:
    def __init__(self, vocab_size, hidden_size):
        self._vocab_size = vocab_size
        self._hidden_size = hidden_size
    def build_graph(self):
        self._X = tf.placeholder(tf.float32, shape = [None, self._vocab_size])
        self._real_Y = tf.placeholder(tf.int32, shape = [None, ])
        
        weights_1 = tf.get_variable(
            name = 'weights_input_hidden',
            shape = (self._vocab_size, self._hidden_size),
            initializer = tf.random_normal_initializer(seed = 2018),
        )
        biases_1 = tf.get_variable(
            name = 'biases_input_hidden',
            shape = (self._hidden_size),
            initializer = tf.random_normal_initializer(seed = 2018),
        )
        weights_2 = tf.get_variable(
            name = 'weights_hidden_output',
            shape = (self._hidden_size, NUM_CLASSES),
            initializer = tf.random_normal_initializer(seed = 2018),
        )
        biases_2 = tf.get_variable(
            name = "biases_hidden_output",
            shape = (NUM_CLASSES),
            initializer = tf.random_normal_initializer(seed = 2018),
        )
        hidden = tf.matmul(self._X, weights_1) + biases_1
        hidden = tf.sigmoid(hidden)
        logits = tf.matmul(hidden, weights_2) + biases_2
        labels_one_hot = tf.one_hot(indices = self._real_Y, depth = NUM_CLASSES, dtype = tf.float32)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels = labels_one_hot, logits = logits)
        loss = tf.reduce_mean(loss)
        probs = tf.nn.softmax(logits)
        predicted_labels = tf.argmax(probs, axis = 1)
        predicted_labels = tf.squeeze(predicted_labels)

        return predicted_labels, loss
 
    def trainer(self, loss, learning_rate):
        train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
        return train_op
   
class DataReader:
    def __init__(self, data_path, batch_size, vocab_size):
        self._batch_size = batch_size
        with open(data_path) as f:
            d_lines = f.read().splitlines()
        self._data = []
        self._labels = []
        for data_id, line in enumerate(d_lines):
            vector = [0.0 for _ in range(vocab_size)]
            features = line.split('<fff>')
            label, doc_id = int(features[0]), int(features[1])
            tokens = features[2].split()
            for token in tokens:
                index, value = int(token.split(':')[0]), float(token.split(':')[1])
                vector[index] = value
            self._data.append(vector)
            self._labels.append(label)
        self._data = np.array(self._data)
        self._labels = np.array(self._labels)
        self._size = len(self._data)
        self._num_epoch = 0
        self._batch_id = 0
    def next_batch(self):
        start = self._batch_id * self._batch_size
        end = start + self._batch_size
        self._batch_id += 1
        if end + self._batch_size > len(self._data):
            end = len(self._data)
            self._num_epoch += 1
            self._batch_id = 0
            indices = list(range(len(self._data)))
            random.seed(2018)
            random.shuffle(indices)
            self._data, self._labels = self._data[indices], self._labels[indices]
        return self._data[start:end], self._labels[start:end]
#Load data
def load_data():
    train_data_reader = DataReader(
    data_path = 'D:/DSLab/DSLab_Training_Phase1/Dataset/20news-bydate/20news-train-tfidf.txt',
    batch_size = 50,
    vocab_size=vocab_size
  )
    test_data_reader = DataReader(
    data_path = 'D:/DSLab/DSLab_Training_Phase1/Dataset/20news-bydate/20news-test-tfidf.txt',
    batch_size = 50,
    vocab_size = vocab_size
  )
    return train_data_reader, test_data_reader

def save_parameters(name, value, epoch):
        filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
        if len(value.shape) == 1: #is a list
            string_form = ','.join([str(number) for number in value])
        else:
            string_form = '\n'.join([','.join([str(number) for number in value[row]]) for row in range(value.shape[0])])
        with open('D:/DSLab/DSLab_Training_Phase1/Dataset/20news-bydate/saved-paras/' + filename, 'w') as f:
            f.write(string_form)
def restore_parameters(name, epoch):
        filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
        with open("D:/DSLab/DSLab_Training_Phase1/Dataset/20news-bydate/saved-paras/" + filename) as f:
            lines = f.read().splitlines()
        if len(lines) == 1:
            value = [float(number) for number in lines[0].split(',')]
        else:
            value = [[float(number) for number in lines[rows].split(',')] for rows in range(len(lines))]
        return value

with open('D:/DSLab/DSLab_Training_Phase1/Dataset/20news-bydate/words_idfs.txt') as f:
    vocab_size = len(f.read().splitlines())
  
NUM_CLASSES = 20

mlp = MLP(vocab_size = vocab_size, hidden_size = 50)

predicted_labels, loss = mlp.build_graph()
train_op = mlp.trainer(loss = loss, learning_rate = 0.1)

train_data_reader, test_data_reader = load_data()

with tf.Session() as sess:
    step, MAX_STEP = 0, 1000

    sess.run(tf.global_variables_initializer())
    while step < MAX_STEP:
        train_data, train_labels = train_data_reader.next_batch()
        plabels_eval, loss_eval, _ = sess.run([predicted_labels, loss, train_op], feed_dict = {mlp._X: train_data, mlp._real_Y: train_labels})
        step += 1
        print('step: {}, loss: {}'.format(step, loss_eval))

        trainable_variables = tf.trainable_variables()
        for variable in trainable_variables:
            save_parameters(name = variable.name, value = variable.eval(), epoch = train_data_reader._num_epoch)

with tf.Session() as sess:
    epoch = train_data_reader._num_epoch

    trainable_variables = tf.trainable_variables()
    for variable in trainable_variables:
        saved_value = restore_parameters(variable.name, epoch)
        assign_op = variable.assign(saved_value)
        sess.run(assign_op)

    num_true_preds = 0
    while True:
        test_data, test_labels = test_data_reader.next_batch()
        test_plabels_eval = sess.run(predicted_labels, feed_dict = { mlp._X: test_data, mlp._real_Y: test_labels})
        matches = np.equal(test_plabels_eval, test_labels)
        num_true_preds += np.sum(matches.astype(float))

        if test_data_reader._batch_id == 0:
            break
      
print('Epoch:{}'.format(epoch))
print('Accuracy on test data:{}'.format(num_true_preds / len(test_data_reader._data)))

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step: 1, loss: 0.6641766428947449
step: 2, loss: 0.0003102297196164727
step: 3, loss: 1.535407932351518e-06
step: 4, loss: 5.483625642455081e-08
step: 5, loss: 4.76837147544984e-09
step: 6, loss: 0.0
step: 7, loss: 21.626018524169922
step: 8, loss: 30.5371150970459
step: 9, loss: 25.27849578857422
step: 10, loss: 18.193992614746094
step: 11, loss: 11.885140419006348
step: 12, loss: 4.511019706726074
step: 13, loss: 1.0964460372924805
step: 14, loss: 0.0067946952767670155
step: 15, loss: 18.916074752807617
step: 16, loss: 21.980440139770508
step: 17, loss: 20.16739273071289
step: 18, loss: 17.66501808166504
step: 19, loss: 14.715853691101074
step: 20, loss: 11.280372619628906
step: 21, loss: 8.379359245300293
step: 

step: 214, loss: 1.1918617486953735
step: 215, loss: 0.9100221395492554
step: 216, loss: 0.9691815972328186
step: 217, loss: 0.7164879441261292
step: 218, loss: 0.9658658504486084
step: 219, loss: 0.7401107549667358
step: 220, loss: 0.794175922870636
step: 221, loss: 0.8878083229064941
step: 222, loss: 0.8653994202613831
step: 223, loss: 0.7796916365623474
step: 224, loss: 1.0862151384353638
step: 225, loss: 0.8265483379364014
step: 226, loss: 0.9711195230484009
step: 227, loss: 1.036349892616272
step: 228, loss: 0.7628336548805237
step: 229, loss: 0.8410715460777283
step: 230, loss: 0.5935696363449097
step: 231, loss: 0.8407084941864014
step: 232, loss: 0.8712412118911743
step: 233, loss: 1.0594398975372314
step: 234, loss: 0.6300300359725952
step: 235, loss: 0.8060867190361023
step: 236, loss: 0.9176609516143799
step: 237, loss: 0.847246527671814
step: 238, loss: 0.9677011370658875
step: 239, loss: 1.169653296470642
step: 240, loss: 0.6495435237884521
step: 241, loss: 0.7127788662910

step: 439, loss: 0.09179869294166565
step: 440, loss: 0.3456719219684601
step: 441, loss: 0.06741172075271606
step: 442, loss: 0.09315009415149689
step: 443, loss: 0.20690101385116577
step: 444, loss: 0.1642235517501831
step: 445, loss: 0.2741878032684326
step: 446, loss: 0.18285278975963593
step: 447, loss: 0.10836867988109589
step: 448, loss: 0.24960510432720184
step: 449, loss: 0.14535969495773315
step: 450, loss: 0.05338016524910927
step: 451, loss: 0.023045353591442108
step: 452, loss: 0.0738467276096344
step: 453, loss: 0.05253177136182785
step: 454, loss: 0.009891383349895477
step: 455, loss: 0.009929139167070389
step: 456, loss: 0.012635987251996994
step: 457, loss: 0.024707332253456116
step: 458, loss: 0.01463110651820898
step: 459, loss: 0.019016383215785027
step: 460, loss: 0.021475594490766525
step: 461, loss: 0.009618332609534264
step: 462, loss: 0.08267613500356674
step: 463, loss: 0.13839298486709595
step: 464, loss: 0.04581601917743683
step: 465, loss: 0.008073158562183

step: 657, loss: 0.0027697542682290077
step: 658, loss: 0.010950446128845215
step: 659, loss: 0.00300815817900002
step: 660, loss: 0.07831110805273056
step: 661, loss: 0.007019403390586376
step: 662, loss: 0.00450220238417387
step: 663, loss: 0.00362885813228786
step: 664, loss: 0.006481174845248461
step: 665, loss: 0.0019053752766922116
step: 666, loss: 0.0035842100623995066
step: 667, loss: 0.0017532057827338576
step: 668, loss: 0.0020501387771219015
step: 669, loss: 0.01833849959075451
step: 670, loss: 0.003580831689760089
step: 671, loss: 0.0022789337672293186
step: 672, loss: 0.0037378696724772453
step: 673, loss: 0.0029292625840753317
step: 674, loss: 0.00433876970782876
step: 675, loss: 0.004101177211850882
step: 676, loss: 0.013980414718389511
step: 677, loss: 0.0020049954764544964
step: 678, loss: 0.0029655920807272196
step: 679, loss: 0.0013799674343317747
step: 680, loss: 0.0011958974646404386
step: 681, loss: 0.0017528739990666509
step: 682, loss: 0.008519329130649567
step:

step: 871, loss: 0.0018143121851608157
step: 872, loss: 0.002906119916588068
step: 873, loss: 0.001244469080120325
step: 874, loss: 0.003525001695379615
step: 875, loss: 0.0015288625145331025
step: 876, loss: 0.0012394734658300877
step: 877, loss: 0.0020825183019042015
step: 878, loss: 0.0007576397038064897
step: 879, loss: 0.000971272645983845
step: 880, loss: 0.0026292740367352962
step: 881, loss: 0.001222278573550284
step: 882, loss: 0.002984805963933468
step: 883, loss: 0.03071279264986515
step: 884, loss: 0.006006630603224039
step: 885, loss: 0.0012730308808386326
step: 886, loss: 0.0009857297409325838
step: 887, loss: 0.0011202696477994323
step: 888, loss: 0.0007854471914470196
step: 889, loss: 0.0010985153494402766
step: 890, loss: 0.002887729788199067
step: 891, loss: 0.0024731471203267574
step: 892, loss: 0.0027556156273931265
step: 893, loss: 0.0009012080845423043
step: 894, loss: 0.0016295632813125849
step: 895, loss: 0.005287406034767628
step: 896, loss: 0.00384244229644536